In [ ]:
import os
import sys
import numpy as np
from scipy import ndimage
from skimage.filters import gaussian
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from skimage import measure  # for marching_cubes (optional 3D surface)
from sklearn.decomposition import PCA
from scipy.ndimage import center_of_mass
from pathlib import Path

In [ ]:
PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
data_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
from library.atlas.atlas_utilities import  adjust_volume, align_masks_numpy, average_images    
from library.controller.sql_controller import SqlController

In [ ]:
def load_volumes(structure):
    images = []
    brains = ['MD585', 'MD589', 'MD594']
    for brain in brains:
        structure_path = os.path.join(data_path, brain, 'structure', f'{structure}.npy')
        arr = np.load(structure_path)
        images.append(arr)
    return images

def plot_slices(volume):
    fig, ax = plt.subplots(1,3,figsize=(9,3))
    ax[0].imshow(volume[volume.shape[0]//2,:,:])
    ax[0].set_title('axial (z)')
    ax[1].imshow(volume[:,volume.shape[1]//2,:])
    ax[1].set_title('coronal (y)')
    ax[2].imshow(volume[:,:,volume.shape[2]//2])
    ax[2].set_title('sagittal (x)')

In [ ]:
structure = 'SC'
masks = load_volumes(structure)

In [ ]:
pca_aligned = align_masks_numpy(masks)
aligned = average_images(masks)

In [ ]:
avg_volume = np.mean(pca_aligned, axis=0)
print(avg_volume.shape, avg_volume.dtype)
allen_id = 255
arr = avg_volume.copy()
z = arr.shape[2] // 2
print('midpoint=',z)
volume = arr.copy()
volume = gaussian(volume, 4.0)            
upper = np.quantile(volume, 0.85)
print('upper', upper)
volume[(volume > upper) ] = allen_id
volume[(volume != allen_id)] = 0
volume = volume.astype(np.uint32)

slice = volume[:,:,z]
plt.imshow(slice, cmap='gray')

In [ ]:
print(aligned.shape, aligned.dtype)
allen_id = 255
arr = aligned.copy()
z = arr.shape[2] // 2
print('midpoint=',z)
volume = arr.copy()
volume = gaussian(volume, 4.0)            
upper = np.quantile(volume, 0.725)
print('upper', upper)
volume[(volume > upper) ] = allen_id
volume[(volume != allen_id)] = 0
volume = volume.astype(np.uint32)

slice = volume[:,:,z]
plt.imshow(slice, cmap='gray')